In [1]:
import pandas as pd
import numpy as np

# Load the CSV file
original_df = pd.read_csv("../data/periodic_backup_data.csv")
# Display the first few rows of the DataFrame
print("Original DataFrame:")
print(original_df.head())

# Create a copy of the original DataFrame that will be expanded
full_df = original_df.copy()

Original DataFrame:
   window-event  window-open  indoor-temp    time  outdoor-temp
0         False         True        73.58  162713         51.08
1         False         True        69.44  157481         54.86
2         False         True        69.08  152994         53.60
3         False         True        73.76  158300         54.14
4         False         True        74.30  158378         52.16


Add and extract features

In [2]:
import sys
if '../' not in sys.path:
    # necessary to access the src folder without relative imports
    sys.path.append('../')
from src.data.prepare_data import *

full_df = add_time_features(original_df)
full_df = add_temp_diff(full_df)
full_df = add_rolling_features(full_df)
full_df = add_lagged_features(full_df)
# Display the first few rows of the DataFrame after feature extraction
print("DataFrame after feature extraction:")
print(full_df.head())

DataFrame after feature extraction:
   window-event  window-open  indoor-temp  outdoor-temp  sin_minutes_of_day  \
0         False         True        73.58         51.08           -0.030539   
1         False         True        69.44         54.86            0.763232   
2         False         True        69.08         53.60            0.999657   
3         False         True        73.76         54.14           -0.422618   
4         False         True        74.30         52.16           -0.095846   

   cos_minutes_of_day  sin_day_of_year  cos_day_of_year  temp_diff  \
0            0.999534         0.936881        -0.349647      22.50   
1           -0.646124         0.953681        -0.300820      14.58   
2            0.026177         0.967938        -0.251190      15.48   
3            0.906308         0.953681        -0.300820      19.62   
4            0.995396         0.953681        -0.300820      22.14   

   rolling_mean_indoor_temp  rolling_std_indoor_temp  \
0           

Visualize feature correlations and perform dimensionality reduction